# Script para limpar os discursos da base:
- remover stopwords
- remover pontuação
- remover caracteres especiais
- remover palavras/termos muito comuns, por exemplo, "Sr. Presidente"

In [1]:
import os
import pandas as pd
import re

import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from unicodedata import normalize


[nltk_data] Downloading package stopwords to /home/zerum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:

def remove_stop_words(data, list_stop_words = None):
    """
    Função para remover stopwords em português
    """
    print('  Remove Stop Words')

    #list_stop_words = set(stopwords.words('portuguese'))
    if list_stop_words != None:
        output = []
        for sentence in data:
            temp_list = []
            for word in sentence.split():
                if word.lower() not in list_stop_words:
                    temp_list.append(word)
            output.append(' '.join(temp_list))

        data = pd.Series(output)    
    
    return data

def remove_punct(data):
    """
    Função para remover pontuações
    """
    print('  Remove Punction')

    puncts = ["\\" +s for s in string.punctuation]
    puncts = '|'.join(puncts)
    data = data.replace(puncts,' ',regex=True)    
    return data

def remove_special_caract(array_data):
    """
    Função para substituir caracteres especiais
    """
    print('  Remove Special Caract')
    data = [normalize('NFKD', data).encode('ASCII', 'ignore').decode('ASCII') for data in array_data]
    return data

def remove_patter_terms(data):
    """
    Função para remover orações que há em todos os discursos, como "senhor presidente", "senhora(s) deputadas"
    """
    #print(data)
    pattern = "^(.+?)-\sSr|(((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?)|\
              (SR\.?\s+?)|(SRA\.?\s+?)|srs|sras|(presidente|deputado(s)|deputada(s))"
    
    data = re.sub(pattern, '',data, flags = re.IGNORECASE)
    
    return data

def clean_text(data, list_stop_words):
    """
    Função para fazer pre-processamento dos textos
    """
    print('Cleaning text...')
    data.isnull().sum()
    data =  data.apply(remove_patter_terms)
    
    data = remove_punct(data)
    data = data.str.lower()
    
    
    list_stop_words = set(list_stop_words)
    data = remove_stop_words(data, list_stop_words)
    
    #data = data.replace('\s+',' ',regex=True)
    
    return data

In [46]:
directory = '/disco01/gabriel/'#"../data/csvs/"
#discursos = pd.read_csv(directory+'discursos_plen.csv')
discursos = pd.read_json(directory+'discursos_plen.json')
discursos.shape

(113143, 13)

In [28]:
discursos.isnull().sum()

FILE                0
ORIGINAL_CONTENT    0
CLEAN_CONTENT       0
SESSION_TYPE        0
SESSION_DATE        0
MONTH               0
YEAR                0
SPEAKER_COUNT       0
Orador              0
Hora                0
ORADOR_CLEAN        0
PARTIDO             0
UF_PARTIDO          0
dtype: int64

In [29]:
discursos.head()

,FILE,ORIGINAL_CONTENT,CLEAN_CONTENT,SESSION_TYPE,SESSION_DATE,MONTH,YEAR,SPEAKER_COUNT,Orador,Hora,ORADOR_CLEAN,PARTIDO,UF_PARTIDO
0,15149-02042001-14h52-PEQUENO EXPEDIENTE-037-3-...,O SR. PAULO PAIM (PT-RS. Sem revisão do orador...,O SR. PAULO PAIM (PT-RS. Sem revisao do orador...,PEQUENO EXPEDIENTE,2001-04-02,4,2001,1,"PAULO PAIM, PT-RS",14h52,PAULO PAIM,PT,RS
1,5628-03102001-17h04-ORDEM DO DIA-190-3-51-O.txt,O SR. GERSON PERES (PPB-PA. Pela ordem. Sem r...,O SR. GERSON PERES (PPB-PA. Pela ordem. Sem r...,ORDEM DO DIA,2001-10-03,10,2001,1,"GERSON PERES, PPB-PA",17h04,GERSON PERES,PPB,PA
2,6562-19092001-16h52-BREVES COMUNICACOES-175-3-...,O SR. JOSÉ GENOÍNO (PT-SP. Sem revisão do orad...,O SR. JOSE GENOINO (PT-SP. Sem revisao do orad...,BREVES COMUNICACOES,2001-09-19,9,2001,1,"JOSÉ GENOÍNO, PT-SP",16h52,JOSÉ GENOÍNO,PT,SP
3,344-18122001-14h00-PEQUENO EXPEDIENTE-001-5-51...,O SR. LINCOLN PORTELA (Bloco/PSL-MG. Pronuncia...,O SR. LINCOLN PORTELA (Bloco/PSL-MG. Pronuncia...,PEQUENO EXPEDIENTE,2001-12-18,12,2001,1,"LINCOLN PORTELA, PSL-MG",14h00,LINCOLN PORTELA,PSL,MG
4,4945-16102001-15h06-GRANDE EXPEDIENTE-203-3-51...,O SR. MARÇAL FILHO (PMDB-MS. Pela ordem. Sem r...,O SR. MARCAL FILHO (PMDB-MS. Pela ordem. Sem r...,GRANDE EXPEDIENTE,2001-10-16,10,2001,1,"MARÇAL FILHO, PMDB-MS",15h06,MARÇAL FILHO,PMDB,MS


In [ ]:
discursos = discursos.dropna()
discursos.index = [i for i in range(0,len(discursos))] #loc[discursos.CLEAN_CONTENT.isna(),:]

In [ ]:
discursos.isnull().sum()

In [47]:
print(len(discursos))
discursos.info()

113143
<class 'pandas.core.frame.DataFrame'>
Int64Index: 113143 entries, 0 to 113142
Data columns (total 13 columns):
FILE                113143 non-null object
ORIGINAL_CONTENT    113143 non-null object
CLEAN_CONTENT       113143 non-null object
SESSION_TYPE        113143 non-null object
SESSION_DATE        113143 non-null object
MONTH               113143 non-null int64
YEAR                113143 non-null int64
SPEAKER_COUNT       113143 non-null int64
Orador              113143 non-null object
Hora                113143 non-null object
ORADOR_CLEAN        113143 non-null object
PARTIDO             113143 non-null object
UF_PARTIDO          113143 non-null object
dtypes: int64(3), object(10)
memory usage: 12.1+ MB


In [ ]:
#discursos = discursos.reindex([i for i in range(0,len(discursos))])#.isnull().sum()

In [ ]:
#discursos.isnull().sum()

In [ ]:
#discursos.head()


In [48]:
# retirando caracteres especiais das stopwords, porque o texto está sem!!
list_stop_words = remove_special_caract(stopwords.words('portuguese'))

# inserindo mais palavras pra ser removidas
list_stop_words.extend(['ser','ja','ha','exa','ainda','ate','todo','todos','toda','todas','devem',
                       'casa','nesta','desta', 'quero'])


discursos.loc[:,'CLEAN_CONTENT'] = clean_text(discursos.loc[:,'CLEAN_CONTENT'],list_stop_words)
#discursos.iloc[57646:57647,:]
print('Finalizado...')


  Remove Special Caract
Cleaning text...
  Remove Punction
  Remove Stop Words
Finalizado...


In [43]:
print(discursos.isna().sum())

FILE             0
CLEAN_CONTENT    0
SESSION_TYPE     0
SESSION_DATE     0
MONTH            0
YEAR             0
SPEAKER_COUNT    0
Hora             0
ORADOR_CLEAN     0
PARTIDO          0
UF_PARTIDO       0
dtype: int64


In [ ]:
print('Qtd de Registros por ANO:\n{}\n\nQtd Total de registros:{} '.format(discursos.YEAR.value_counts(),len(discursos)))

In [49]:
discursos.head()

,FILE,ORIGINAL_CONTENT,CLEAN_CONTENT,SESSION_TYPE,SESSION_DATE,MONTH,YEAR,SPEAKER_COUNT,Orador,Hora,ORADOR_CLEAN,PARTIDO,UF_PARTIDO
0,15149-02042001-14h52-PEQUENO EXPEDIENTE-037-3-...,O SR. PAULO PAIM (PT-RS. Sem revisão do orador...,ocupo tribuna registrar protocolando junto com...,PEQUENO EXPEDIENTE,2001-04-02,4,2001,1,"PAULO PAIM, PT-RS",14h52,PAULO PAIM,PT,RS
1,5628-03102001-17h04-ORDEM DO DIA-190-3-51-O.txt,O SR. GERSON PERES (PPB-PA. Pela ordem. Sem r...,objecao desde cidadao apresente provas incrimi...,ORDEM DO DIA,2001-10-03,10,2001,1,"GERSON PERES, PPB-PA",17h04,GERSON PERES,PPB,PA
2,6562-19092001-16h52-BREVES COMUNICACOES-175-3-...,O SR. JOSÉ GENOÍNO (PT-SP. Sem revisão do orad...,continuamos defendendo posicao governo forcas ...,BREVES COMUNICACOES,2001-09-19,9,2001,1,"JOSÉ GENOÍNO, PT-SP",16h52,JOSÉ GENOÍNO,PT,SP
3,344-18122001-14h00-PEQUENO EXPEDIENTE-001-5-51...,O SR. LINCOLN PORTELA (Bloco/PSL-MG. Pronuncia...,poderia deixar lamentar desmando governo feder...,PEQUENO EXPEDIENTE,2001-12-18,12,2001,1,"LINCOLN PORTELA, PSL-MG",14h00,LINCOLN PORTELA,PSL,MG
4,4945-16102001-15h06-GRANDE EXPEDIENTE-203-3-51...,O SR. MARÇAL FILHO (PMDB-MS. Pela ordem. Sem r...,parlamentares vez louvo iniciativa governo fed...,GRANDE EXPEDIENTE,2001-10-16,10,2001,1,"MARÇAL FILHO, PMDB-MS",15h06,MARÇAL FILHO,PMDB,MS


In [50]:
discursos_final = discursos.drop(columns = ['ORIGINAL_CONTENT','Orador'])

In [60]:
discursos_final.columns = discursos_final.columns.to_series().apply(lambda x: x.upper())
discursos_final.columns

Index(['FILE', 'CLEAN_CONTENT', 'SESSION_TYPE', 'SESSION_DATE', 'MONTH',
       'YEAR', 'SPEAKER_COUNT', 'HORA', 'ORADOR_CLEAN', 'PARTIDO',
       'UF_PARTIDO'],
      dtype='object')

In [61]:
## salvando dados limpos em uma nova base
#discursos.to_csv(directory+'discursos_plen_limpo.csv', index = False)
discursos_final.to_json(directory+'discursos_plen_limpo.json')

In [62]:
print(140901 - 140997,
      138562 - 139021)


-96 -459
